# AI cập nhật dữ liệu mới nhất cho bản tin 14h00

In [ ]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import import_gemini
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload
import other_source_data

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(import_gemini)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)
importlib.reload(other_source_data)

from import_default import *
from import_database import *
from import_other import *
from import_gemini import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *
from other_source_data import *

### Phần dữ liệu lịch sử các ticker

#### Dữ liệu từ Mongo

In [ ]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

if date_series['date'].max() < get_today_date():
    new_date = get_today_date()
    date_series = pd.concat([date_series, pd.DataFrame({'date': [new_date]})]).sort_values(by='date', ascending=False).reset_index(drop=True)

In [ ]:
projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1,'volume': 1}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

itd_index_df = get_mongo_collection(stock_db, "itd_index")
other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

#### Chuẩn bị các dữ liệu cần thiết

In [ ]:
itd_data_list = []
for ticker in ['VNINDEX', 'VN30', 'HNXINDEX', 'UPINDEX', 'VN30F1M', 'VN30F2M']:
    temp_itd_df = itd_index_df[itd_index_df['ticker'] == ticker].copy()
    temp_eod_df = full_index_df[full_index_df['ticker'] == ticker].reset_index(drop=True)
    temp_close_prev = temp_eod_df[temp_eod_df['date'] < temp_itd_df['date'].iloc[-1].normalize()].iloc[0]['close'].item()

    temp_itd_df['pct_change'] = (temp_itd_df['close'] - temp_close_prev) / temp_close_prev
    temp_itd_df['diff'] = temp_itd_df['close'] - temp_close_prev

    # Chuyển cột date về datetime và làm index
    temp_itd_df['date'] = pd.to_datetime(temp_itd_df['date'])
    temp_itd_df = (
        temp_itd_df
        .set_index('date')
        .resample('5T')
        .agg({
            'ticker': 'first',  # Giữ nguyên mã chứng khoán
            'open':  'first',
            'high':  'max',
            'low':   'min',
            'close': 'last',
            'volume':'sum',
            'pct_change': 'last',
            'diff': 'last'
        })
        .dropna(subset=['open'])  # bỏ các khung thời gian không có dữ liệu
        .reset_index()
    )
    itd_data_list.append(temp_itd_df)

daily_14h00_data_df = pd.concat(itd_data_list, ignore_index=True)

### Phần dữ liệu cho tin tức

#### Khởi tạo model và danh sách link báo

In [ ]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

In [ ]:
article_url_dict = {
    'VietStock': {
        'https://vietstock.vn/775/the-gioi/kinh-te-nganh.rss': 3,
        'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss': 3,
        'https://vietstock.vn/830/chung-khoan/co-phieu.rss': 3,
        'https://vietstock.vn/761/kinh-te/vi-mo.rss': 3,
        'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss': 3,
        'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss': 3,
        'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss': 3,
    },
    'CafeF': {
        'https://cafef.vn/vi-mo-dau-tu.chn': 5,
        'https://cafef.vn/thi-truong-chung-khoan.chn': 5,
        'https://cafef.vn/tai-chinh-quoc-te.chn': 5,
        'https://cafef.vn/doanh-nghiep.chn': 5,
    },
    'Vietnambiz': {
        'https://vietnambiz.vn/tai-chinh.htm': 5,
        'https://vietnambiz.vn/quoc-te.htm': 5,
        'https://vietnambiz.vn/chung-khoan.htm': 5,
        'https://vietnambiz.vn/doanh-nghiep.htm': 5,
    },
    'VnEconomy': {
        'https://vneconomy.vn/kinh-te-the-gioi.rss': 5,
        'https://vneconomy.vn/tai-chinh.rss': 5,
        'https://vneconomy.vn/chung-khoan.rss': 5,
    }
}

#### Lấy các bài báo từ website

In [ ]:
# 1. Tạo dictionary cấu hình để quản lý cả 2 pattern xử lý
source_handlers = {
    'VnEconomy': {
        'type': 'batch',
        'process_batch': get_article_vneconomy 
    },
    'VietStock': {
        'type': 'item_by_item',
        'get_articles': lambda url, num: feedparser.parse(url).entries[:num],
        'get_details': lambda entry: get_article_vietstock(entry['id']),
        'get_published_time': lambda entry: getattr(entry, 'published', '') or ''
    },
    'CafeF': {
        'type': 'item_by_item',
        'get_articles': get_cafef_articles_list,
        'get_details': lambda entry: get_article_cafef(entry['id']),
        'get_published_time': lambda entry: get_cafef_published_time(entry['id'])
    },
    'Vietnambiz': {
        'type': 'item_by_item',
        'get_articles': get_vietnambiz_articles_list,
        'get_details': lambda entry: get_article_vietnambiz(entry['id']),
        'get_published_time': lambda entry: get_vietnambiz_published_time(entry['id'])
    }
}

# 2. Vòng lặp xử lý chính, giờ đây đã trở nên gọn gàng
raw_news_list = []
for source, rss_list in article_url_dict.items():
    for rss_url, num_articles in rss_list.items():
        
        handler = source_handlers.get(source)
        if not handler:
            print(f"Warning: No handler found for source '{source}'. Skipping.")
            continue

        # Phân luồng xử lý dựa trên 'type' đã định nghĩa trong handler
        if handler['type'] == 'batch':
            # Xử lý các nguồn trả về một danh sách hoàn chỉnh
            temp_news_list = handler['process_batch'](rss_url, num_articles)
            raw_news_list.extend(temp_news_list) # Dùng extend hiệu quả hơn là +

        elif handler['type'] == 'item_by_item':
            # Xử lý các nguồn cần lấy chi tiết từng tin
            feed_entries = handler['get_articles'](rss_url, num_articles)
            for entry in feed_entries:
                content, image_url = handler['get_details'](entry)
                published_time = handler['get_published_time'](entry)
                
                raw_news_list.append({
                    'source': source,
                    'title': entry['title'], 
                    'content': content,
                    'image_url': image_url,
                    'article_url': entry['id'],
                    'published_time': published_time,
                })

# 3. Chuyển đổi danh sách tin thành DataFrame
raw_news_df = pd.DataFrame(raw_news_list)
raw_news_df['published_time'] = raw_news_df['published_time'].apply(convert_published_time)

#### Lọc, tóm tắt và chọn tin nổi bật

In [ ]:
#Lọc bớt các tin có tiêu đề dài > 20 từ
raw_news_df = raw_news_df[(raw_news_df['title'].str.split().str.len() <= 20)].reset_index(drop=True)

# Lọc và phân loại các tin nổi bật
fithered_news_df = raw_news_df.copy()
filtered_news_index_dict = get_filtered_news_index(standard_model_dict, fithered_news_df, num_articles=10)

# Tạo dictionary ánh xạ từ tên nhóm đến index của các tin nổi bật
news_type_map = {}
for k, idx_list in filtered_news_index_dict.items():
    for idx in idx_list:
        news_type_map[idx] = k

# Thêm cột news_type vào fithered_news_df và lọc ra các dòng có news_type
fithered_news_df['news_type'] = fithered_news_df.index.map(news_type_map)
fithered_news_df = fithered_news_df[fithered_news_df['news_type'].notnull()]
fithered_news_df = fithered_news_df.sort_values(by=['news_type', 'published_time'], ascending=[False, False]).reset_index(drop=True)

In [ ]:
%%capture
# Tạo danh sách tin hàng tuần với nội dung tóm tắt
daily_8h30_news_df = fithered_news_df.copy()
daily_8h30_news_df['content'] = daily_8h30_news_df['content'].apply(lambda x: summary_daily_article(fast_model_dict, x))
daily_8h30_news_df['title'] = daily_8h30_news_df.apply(lambda x: create_new_article_title(fast_model_dict, x['title'], x['content']), axis=1)

# Thêm các cột cần thiết
daily_8h30_news_df['word_count'] = daily_8h30_news_df['content'].str.split().str.len()
daily_8h30_news_df['impact'] = analyze_news_impact(standard_model_dict, daily_8h30_news_df)

In [ ]:
# Lựa chọn 1 tin tức nổi bật nhất cho mỗi nhóm
daily_14h00_news_df['major_selected'] = identify_major_selected(standard_model_dict, daily_14h00_news_df)

# Lựa chọn 3 tin tức phụ cho mỗi nhóm
top_news_index_list = []
for news_type in daily_14h00_news_df['news_type'].unique():
    temp_news_index_list = get_daily_top_news(standard_model_dict, daily_14h00_news_df, news_type, num_articles = 3)
    top_news_index_list = top_news_index_list + temp_news_index_list
daily_14h00_news_df['sub_selected'] = daily_14h00_news_df.index.isin(top_news_index_list)
daily_14h00_news_df['sub_selected'] = daily_14h00_news_df['sub_selected'].apply(lambda x: 'x' if x else '')

# Gộp các tin được chọn vào cột ai_selected
daily_14h00_news_df['ai_selected'] = daily_14h00_news_df.apply(lambda x: 'o' if x['major_selected'] == 'x' else ('x' if x['sub_selected'] == 'x' else ''), axis=1)
daily_14h00_news_df = daily_14h00_news_df.drop(columns=['major_selected', 'sub_selected'])

# Sắp xếp lại các cột
daily_14h00_news_df['impact'] = pd.Categorical(daily_14h00_news_df['impact'], categories=['TÍCH CỰC', 'TRUNG LẬP', 'TIÊU CỰC'], ordered=True)
daily_14h00_news_df.sort_values(by=['news_type', 'impact'], ascending=[False, True], inplace=True)

### Lưu dữ liệu vào MSSQL

In [ ]:
daily_14h00_time_df = pd.DataFrame([{
    'time': time_series.iloc[0].item().strftime('%H:%M'),
    'date': time_series.iloc[0].item().strftime('%d/%m/%Y')
}])

In [ ]:
%%capture
save_to_mssql(cts_engine, daily_14h00_data_df, 'daily_14h00_data')
save_to_mssql(cts_engine, daily_14h00_news_df, 'daily_14h00_news')
save_to_mssql(cts_engine, daily_14h00_time_df, 'daily_14h00_time')
